# Web-Scraping - Informationen von Websites downloaden
© Thomas Robert Holy 2019
<br>
Version 1.0
<br><br>
Visit me on GitHub: https://github.com/trh0ly

## Grundlegende Einstellungen:
### Import von Modulen
Zunächst müssen die notwendigen Module importiert werden, damit auf diese zugegriffen werden kann. 

In [ ]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import string
import numpy as np
import requests

## Prüfung der Websites
Nun müssen die Websites im Ranking systematisch überprüft werden, da einzelne Seiten zwischendurch nicht aufrufbar sind.

In [ ]:
#-----------------------------------------------------------------
# Prüfe die Website auf Verfügbarkeit und speichere die verfügbaren
# "Schlüsselzahlen" in der Liste "good_pages"

#----------------------------------------
# Definition der Funktion zur Seiten-Prüfung

good_pages = []
def find_good_pages(url_addon):
    url = "https://ranking.zeit.de/che/de/ort/" + str(url_addon)
    try:
        good_page = urlopen(url)
        good_pages.append(url_addon)
    except:
        pass

print("Please Wait.. The url check will take some time")

#----------------------------------------
# Führe die Aktion für Seite 1 bis 350 durch

for i in range(1,430+1):
    find_good_pages(i)
    if i == 430:
        print('Ready')
print('The "good_pages" are: ' + str(good_pages))

## Download der Informationen und Speicherung
Im nächsten Schritt werden die validen Websites nach Informationen durchsucht.
Diese werden dann anschließend erschlossen und aufbereitet.
Zum Schluss werden die so extrahierten Informationen im einer Text.Datei gespeichert. 

In [ ]:
print('Working on the next Step... \n')

#-----------------------------------------------------------------
# Extraktion der relevanten Daten aus den jeweiligen sEiten

#----------------------------------------
# Öffne das File in dem die Daten gespeichert
# werden sollen

text_file = open("output.txt","w")

#----------------------------------------
# Extraktionsvorgang für jede "gute" Seite

for good_page in sorted(good_pages):

    # Aktuelle url definieren
    url = "https://ranking.zeit.de/che/de/ort/" + str(good_page)
    print('Aktuelle URL: ' + str(url))

    # Öffne die url
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'lxml')

    # Lade den Titel zur url
    title = soup.title
    print('Aktueller Titel: ' + str(title))

    # Scanne nach Tabellen 
    rows = soup.find_all('table')
    #print(rows)

    # Wandle die Tabelleninhalte in einen String um
    # und extrahiere den Text
    str_cells = str(rows)
    cleanstring = BeautifulSoup(str_cells, "lxml").get_text()
    #print(cleanstring)

    #----------------------------------------
    # Enferne störende Zeichen

    cleanstring = cleanstring.replace('%', '')
    cleanstring = cleanstring.replace(']', '')
    cleanstring = cleanstring.replace('[', '')
    cleanstring = cleanstring.replace('*', '')
    cleanstring = cleanstring.replace('\r', '')
    cleanstring = cleanstring.replace(" ,", '')
    cleanstring = cleanstring.replace(", ", '')
    cleanstring = cleanstring.replace(" ", '')
    cleanstring = cleanstring.replace("€", '')

    #----------------------------------------
    # Separiere einzelne Wörter und Nummern nun
    # in separate Strings und speichere diese in 
    # einer Liste

    cleaned_string = re.sub('['+string.punctuation+']', '', cleanstring).split() 
    #print(cleaned_string)

    # Extrahiere alle Zahlen aus dieser Liste und speichere sie in 
    # der Liste "pure_zahlen"    
    _re_digits = re.compile(r"(-?(?:(?:\d+(?:\.\d*)?)|(?:\.\d+)))")
    pure_zahlen = []
    for element in cleaned_string:
        pure_zahlen += [ float(n) for n in _re_digits.findall(element)]
    #print(pure_zahlen)

    # Extrahiere alle Wörter aus dieser Liste und speichere sie in 
    # der Liste "pure_text"    
    pure_text = [x for x in cleaned_string if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
    #print(pure_text)
    
    # Erstelle ein Dictonary das den beiden Keywords 'Kategorie' und 'Wert' die 
    # entsprechenden Listen zuordnet, welches grundlage für den DataFrame "table"
    # ist in welchem alle Informationen in tabellenfrom aufgeführt werden. 
    # Nicht vorhandene Einträge werden mit "NaN" gekennzeichnet
    my_dict = {'Kategorie': np.array(pure_text), 'Wert': np.array(pure_zahlen)}
    table = pd.concat([pd.Series(v, name=k) for k, v in my_dict.items()], axis=1)
    #print(table)
    #print('\n')
    
    # Speichere die relevanten Inoformationen mit ggf. 
    # Umbrüchen in dem File
    text_file.write(str(title))
    text_file.write(str(url))
    text_file.write('\n')
    text_file.write(str(table))
    text_file.write('\n')
    text_file.write('\n')    

#----------------------------------------
# File schließen

text_file.close()